<a href="https://colab.research.google.com/github/MarcChen/Fewshot-ICL/blob/main/CHEN_Marc_SEBTI_Adam_TableGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: TableGPT


In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections:
0. Installation: This section is dedicated to module installation, model loading and data loading.
1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.


IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>


<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B

###Q1 : What are the specificity of TableGPT2

TableGPT2 is a model designed to handle tabular data and generate code or insights for data analysis tasks. It processes tables formatted such as DataFrames and provides text outputs, ideal for generating analysis scripts. It's built on Qwen2.5 and it has been fine-tuned with millions of examples.

---

## 0. Setup

In [11]:
!pip install transformers datasets bitsandbytes accelerate > \dev\null

In [12]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import pandas as pd
import torch

In [6]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
)
# Set LLM on eval mode.
llm.eval()

model-00001-of-00004.safetensors:   3%|2         | 136M/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-0

In [7]:
generation_config = GenerationConfig(
  max_new_tokens=512,
  do_sample=False,
  # do_sample=True,
  # temperature=.7,
  # top_p=.8,
  # top_k=20,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [8]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


------

## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:
1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [9]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

def generate_answer(prompt, llm=llm, generation_config=generation_config):
  # Code taken in the quickstart of TableGPT2 https://huggingface.co/tablegpt/TableGPT2-7B

  messages = [
    {"role": "system", "content": "You are a helpful assistant to write some python code to solve problems on pandas dataframe."},
    {"role": "user", "content": prompt},
  ]

  text = tokenizer.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)

  generated_ids = llm.generate(**model_inputs, max_new_tokens=512, generation_config=generation_config)
  generated_ids = [
      output_ids[len(input_ids) :]
      for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return response




prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Hen

### Test of the provided output thanks to TableGPT2 answer given abbove this cell

In [10]:
import pandas as pd

# Assuming the dataframe is named 'df'
# Filter the dataframe to include only passengers under 18 who survived
child_survivors = df[(df['Age'] < 18) & (df['Survived'] == 1)]

# Count the number of child survivors
num_child_survivors = child_survivors.shape[0]

# Output the result
output = num_child_survivors
print(output)

61


## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

```
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
```

So we will need to execute it, but there is some difficulty:
1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [13]:
import re

In [14]:
def exec_answer(answer, gold, debug=False):
    try:
        # Extract the code from the answer. Be careful, the code is now always in ``` ```.
        reg = re.search(r"```python(.*?)```", answer, re.DOTALL)
        code = answer if not reg else reg.group(1) # Handling the case when llm directly provide the answer

        local_vars = {'df': df}
        # Execute the code, https://docs.python.org/3/library/functions.html#exec
        exec(code, {}, local_vars)

        # if the code work: Return True or False based on output == gold (be careful to handle iterable !)
        print(f"Generated output : {local_vars.get('output')}") if debug else None
        print(f"Gold output : {gold}") if debug else None
        return local_vars.get("output") == gold
    except Exception as e:
        # If the code fails to execute then we return False
        print(f"\033[93mExec answer function failed - exception {e} \033[0m")
        return False

print(exec_answer(answer, 61))

61
True


## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


### Question we ask : What's the maximum value of {choosen_col} where {filter_col} == {filter_value}?

In [15]:
import random

def generate_random_question(generate_function, df, k=1, seed=42):
  random.seed(seed)
  return [generate_function(df) for _ in range(k)]

def generate_filter_question(df):
    # Create a question template that takes a target column, a filter column, and a filter value
    numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
    columns = df.columns.tolist()
    choosen_col = random.choice(numeric_columns)
    filter_col = random.choice(columns)

    # Target & Filter columns should be different
    while filter_col == choosen_col:
        filter_col = random.choice(columns)

    # Get a random filter value inside the filter column. Avoid NaN values.
    not_nan = df[filter_col].dropna().unique()
    filter_value = random.choice(not_nan)

    # Compute the correct answer for the given target column, filter column, and filter value.
    filtered_df = df[df[filter_col] == filter_value]
    answer = None if filtered_df.empty else filtered_df[choosen_col].max()

    # Return formatted question and associated answer in a dict {"question": [question], "answer": [answer]}
    question_formated = (f"What's the maximum value of {choosen_col} where {filter_col} == {filter_value}?")

    return {"question": question_formated, "answer": answer}

generate_random_question(generate_filter_question, df, k=5)

[{'question': "What's the maximum value of Parch where Survived == 0?",
  'answer': 6},
 {'question': "What's the maximum value of Parch where Sex == male?",
  'answer': 5},
 {'question': "What's the maximum value of Survived where Pclass == 2?",
  'answer': 1},
 {'question': "What's the maximum value of PassengerId where Embarked == Q?",
  'answer': 891},
 {'question': "What's the maximum value of SibSp where Survived == 1?",
  'answer': 4}]

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


### It kept creating new dataframe for example as if the dataframe given in the prompt didn't exist. So we emphasized a bit more in the prompt on the existing dataframe named {var_name}

In [32]:
prompt_template = """Given access to pandas dataframe named {var_name} which already exist, write the Python code to answer the user's question.
The answer should be store in a variable named "output". Here is a sample of the existing dataframe named {var_name}:

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

In [17]:
from tqdm import tqdm

num_question = 20
correct = 0

questions = generate_random_question(generate_filter_question, df, k=num_question)

# Iterate over question to format prompt, generate answer and execute answer.
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    # print(f"Expected answer {expected_answer}")
    # print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question=question,
)

    generated_answer = generate_answer(prompt)
    # print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")

# Report the accuracy
accuracy = correct / len(questions)
print(f"\033[91mAccuracy : {accuracy * 100 :.2f}\033[0m")


  5%|▌         | 1/20 [00:09<02:55,  9.22s/it]

 Correctly answered 


 10%|█         | 2/20 [00:15<02:16,  7.56s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:22<02:06,  7.44s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [03:27<00:08,  8.72s/it]

 Correctly answered 


100%|██████████| 20/20 [03:35<00:00, 10.79s/it]

 Correctly answered 
Accuracy : 80.00


---

## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


# Question generated is now :     question = "How many distinct values of {choosen_col} where {filter_col} == {filter_value}?"


In [18]:
def generate_filter_count_distinct_question(df):
    # Get a random target column and filter column
    columns = df.columns.tolist()
    choosen_col = random.choice(columns)
    filter_col = random.choice(columns)

    while filter_col == choosen_col:
        filter_col = random.choice(columns)

    # Get a random filter value inside the filter column. Avoid NaN values.
    not_nan = df[filter_col].dropna().unique()

    filter_value = random.choice(not_nan)

    # Compute the correct answer: count distinct values in the target column that meet the condition
    filtered_df = df[df[filter_col] == filter_value]
    answer = filtered_df[choosen_col].nunique()

    # Return formatted question and associated answer in a dict {"question": [question], "answer": [answer]}
    question = (f"How many distinct values of {choosen_col} where {filter_col} == {filter_value}?")

    return {"question": question, "answer": answer}


generate_random_question(generate_filter_count_distinct_question, df, k=5)

[{'question': 'How many distinct values of Embarked where Survived == 0?',
  'answer': 3},
 {'question': 'How many distinct values of Sex where Name == Stankovic, Mr. Ivan?',
  'answer': 1},
 {'question': 'How many distinct values of Pclass where Survived == 0?',
  'answer': 3},
 {'question': 'How many distinct values of Fare where SibSp == 1?',
  'answer': 94},
 {'question': 'How many distinct values of PassengerId where Survived == 0?',
  'answer': 424}]

In [40]:
questions = generate_random_question(generate_filter_count_distinct_question, df, k=num_question)

# Iterate over questions to format prompt, generate answer, and compute accuracy
correct = 0
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    # print(f"Expected answer {expected_answer}")
    # print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question=question,
)

    generated_answer = generate_answer(prompt)
    # print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")
    else:
        print(f"\033[91m Incorrectly answered \033[0m")
# Report the accuracy
accuracy = correct / len(questions)
print(f"\033[91mAcc: {accuracy * 100 :.2f}\033[0m")

  5%|▌         | 1/20 [00:07<02:30,  7.91s/it]

 Correctly answered 


 10%|█         | 2/20 [00:15<02:23,  7.99s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:23<02:12,  7.82s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [02:46<00:07,  7.42s/it]

 Correctly answered 


100%|██████████| 20/20 [02:54<00:00,  8.70s/it]

 Correctly answered 
Acc: 95.00


## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>

In [20]:
adult_df = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult_df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [21]:
# Rewrote this function so we can use it with adult_df as previous function didn't have df as a parameter
def exec_answer_adult_df(answer, gold, debug=False):
    try:
        # Extract the code from the answer. Be careful, the code is now always in ``` ```.
        reg = re.search(r"```python(.*?)```", answer, re.DOTALL)
        if not reg:
            code = answer
        else :
          code = reg.group(1)

        local_vars = {'adult_df': adult_df}
        # Execute the code, https://docs.python.org/3/library/functions.html#exec
        #print(code)
        exec(code, {}, local_vars)

        # if the code work: Return True or False based on output == gold (be careful to handle iterable !)

        print(f"Generated output : {local_vars.get('output')}") if debug else None
        print(f"Gold output : {gold}") if debug else None
        return local_vars.get("output") == gold
    except Exception as e:
       # If the code fails to execute then we return False
        print(f"\033[93mExec answer function failed - exception {e} \033[0m")
        return False

In [22]:
prompt_template = """Given access to pandas dataframe named {var_name} which already exist, write the Python code to answer the user's question.
The answer should be store in a variable named "output". Here is a sample of the existing dataframe named {var_name}:

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

In [23]:
num_question = 20
correct = 0

questions = generate_random_question(generate_filter_count_distinct_question, adult_df, k=num_question)

# Iterate over questions to format prompt, generate answer, and compute accuracy
correct = 0
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    #print(f"Expected answer {expected_answer}")
    #print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="adult_df",
    df_info=adult_df.head(5).to_string(index=False),
    user_question=question,
)


    generated_answer = generate_answer(prompt)
    #print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer_adult_df(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")


# Report the accuracy
accuracy = correct / len(questions)
print(f"\n\033[91mAccuracy : {accuracy * 100 :.2f}\033[0m")

  5%|▌         | 1/20 [00:07<02:19,  7.33s/it]

 Correctly answered 


 10%|█         | 2/20 [00:16<02:28,  8.23s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:26<02:36,  9.21s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [02:31<00:07,  7.50s/it]

 Correctly answered 


100%|██████████| 20/20 [03:13<00:00,  9.67s/it]


Accuracy : 95.00


In [24]:
num_question = 20
correct = 0

questions = generate_random_question(generate_filter_question, adult_df, k=num_question)

# Iterate over questions to format prompt, generate answer, and compute accuracy
correct = 0
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    #print(f"Expected answer {expected_answer}")
    #print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="adult_df",
    df_info=adult_df.head(5).to_string(index=False),
    user_question=question,
)


    generated_answer = generate_answer(prompt)
    #print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer_adult_df(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")


# Report the accuracy
accuracy = correct / len(questions)
print(f"\n\033[91mAccuracy : {accuracy * 100 :.2f}\033[0m")

  5%|▌         | 1/20 [00:07<02:17,  7.22s/it]

 Correctly answered 


 10%|█         | 2/20 [00:14<02:15,  7.54s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:23<02:18,  8.14s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [02:25<00:07,  7.60s/it]

 Correctly answered 


100%|██████████| 20/20 [02:33<00:00,  7.66s/it]

 Correctly answered 

Accuracy : 100.00


We had very good accuracies for both prompts. The results are similar to the ones on the titanic table, the model is consistant in it's performance.

## Bonus

In [78]:
titanic_df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
titanic_df = titanic_df.drop("Cabin", axis=1).dropna()
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


In [95]:
prompt_template = """Given access to multiple pandas DataFrames named {var_name1} and {var_name2} which already exist, write the Python code to answer the user's question using BOTH dataframe.
The answer should be store in a variable named "output". Here is a sample of the existing dataframe named {var_name1} and {var_name2}:

/*
"{var_name1}.head(5).to_string(index=False)" is as follows:
{df_info1}

"{var_name2}.head(5).to_string(index=False)" is as follows:
{df_info2}
*/

Question: {user_question}
"""

In [96]:
prompt = prompt_template.format(
    var_name1="titanic_df",
    df_info1=titanic_df.head(5).to_string(index=False),
    var_name2="adult_df",
    df_info2=adult_df.head(5).to_string(index=False),
    user_question="What is the mean salary of titanic survivors based on the adult dataset income?",
)

generated_answer = generate_answer(prompt)

In [97]:
reg = re.search(r"```python(.*?)```", generated_answer, re.DOTALL)
code = generated_answer if not reg else reg.group(1)

In [98]:
print(f"Generated code is :\n----\n{code}\n\n----\nPrompt was\n----\n\n{prompt}")

Generated code is :
----

import pandas as pd

# Filter the titanic_df to include only the survivors
survivors_df = titanic_df[titanic_df['Survived'] == 1]

# Merge the filtered titanic_df with the adult_df based on the income column
merged_df = pd.merge(survivors_df, adult_df, left_on='Survived', right_on='income')

# Calculate the mean salary of the merged dataframe
mean_salary = merged_df['capital-gain'].mean()

output = mean_salary


----
Prompt was
----

Given access to multiple pandas DataFrames named titanic_df and adult_df which already exist, write the Python code to answer the user's question using BOTH dataframe.
The answer should be store in a variable named "output". Here is a sample of the existing dataframe named titanic_df and adult_df:

/*
"titanic_df.head(5).to_string(index=False)" is as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3 

In [99]:
local_vars = {'adult_df': adult_df,
              'titanic_df': titanic_df}
exec(code, {}, local_vars)
print(local_vars.get("output"))

ValueError: You are trying to merge on int64 and object columns for key 'Survived'. If you wish to proceed you should use pd.concat

- We tried different outputs to get the mean salary of titanic surviror based on adult dataset. We added in the prompt a hint saying that the model should leverage common ground between dataframes and still it fails to generate a correct answer.

- We encoutered 3 different model's reactions:
  - The model refuses to generate the query saying it's impossible as the are no direct links
  - It often uses non matching keys to perform joins
  - The generated code does nothing useful and the result is quite often "Nan"
  - We encouter syntax error : the model focuses too much on the dataframes and less on the code quality

---
*I'll be honset at first I was a bit confused on how to do it but I guess we should make a hypothesis that for those who survived there shoudln't be much overlapping combination of age + sexe so we could use that as a key to perform the join*